In [7]:
import cv2
import mediapipe as mp

In [8]:
# Initialize Mediapipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

In [9]:
# Define the virtual button region (x1, y1, x2, y2)
# button_region = (100, 100, 200, 200)

In [10]:
# Capture video from the webcam
cap = cv2.VideoCapture(0)

In [11]:
# Initialize state variables
led_on = False
fan_on = False
led_pressed = False
fan_pressed = False

In [12]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Define the virtual button region to cover the whole screen
    button_region = (0, 0, frame.shape[1], frame.shape[0])

    # Process the frame and detect hands
    results = hands.process(frame_rgb)

    # Draw the virtual button region
    cv2.rectangle(frame, (button_region[0], button_region[1]), (button_region[2], button_region[3]), (0, 255, 0), 2)

    count = 0

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the coordinates of the index finger tip (landmark 8)
            x_index = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * frame.shape[1])
            y_index = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * frame.shape[0])

            # Get the coordinates of the middle finger tip (landmark 12)
            x_middle = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * frame.shape[1])
            y_middle = int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * frame.shape[0])

            # Check if the index fingertip is within the button region and pointing upwards
            if (button_region[0] < x_index < button_region[2] and button_region[1] < y_index < button_region[3] and
                y_index < int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y * frame.shape[0]) and
                y_index < int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y * frame.shape[0])):
                count += 1

            # Check if the middle fingertip is within the button region and pointing upwards
            if (button_region[0] < x_middle < button_region[2] and button_region[1] < y_middle < button_region[3] and
                y_middle < int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y * frame.shape[0]) and
                y_middle < int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y * frame.shape[0])):
                count += 1

    # Toggle the state based on the number of fingers inside the button region
    if count == 1:
        if not led_pressed:
            led_on = not led_on
            led_pressed = True
    else:
        led_pressed = False

    if count == 2:
        if not fan_pressed:
            fan_on = not fan_on
            fan_pressed = True
    else:
        fan_pressed = False

    # Display the current state of LED and FAN
    led_state = "LED on" if led_on else "LED off"
    fan_state = "FAN on" if fan_on else "FAN off"
    cv2.putText(frame, led_state, (5, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, fan_state, (5, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('Virtual Button', frame)
    cv2.moveWindow('Virtual Button', 0, 0)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()